In [ ]:
import json
import requests
from collections import Counter

DEEPSEEK_API_URL = "https://api.deepseek.com/v1/chat/completions" 
API_KEY = "YOUR_DEEPSEEK_API_KEY"  # Replace with your DeepSeek API key

def analyze_sentiment_with_llm(text: str) -> str:
    """
    Analyze sentiment using DeepSeek's LLM.
    """
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "deepseek-chat",  
        "messages": [
            {
                "role": "system",
                "content": "You are a sentiment analysis assistant. Analyze the sentiment of the following text and respond with 'bullish', 'bearish', or 'neutral'."
            },
            {
                "role": "user",
                "content": f"Analyze the sentiment of this text: {text}"
            }
        ],
        "max_tokens": 10,
        "temperature": 0.0
    }
    response = requests.post(DEEPSEEK_API_URL, headers=headers, json=data)
    if response.status_code == 200:
        
        sentiment = response.json()["choices"][0]["message"]["content"].strip().lower()
        if sentiment in ["bullish", "bearish", "neutral"]:
            return sentiment
        else:
            return "neutral"  
    else:
        print(f"Error analyzing sentiment: {response.status_code}")
        return "neutral"


def calculate_sentiment_percentages(sentiment_counts: Counter, total_items: int) -> dict:
    """
    Calculate the percentage of each sentiment.
    """
    return {
        "bullish": (sentiment_counts["bullish"] / total_items) * 100,
        "bearish": (sentiment_counts["bearish"] / total_items) * 100,
        "neutral": (sentiment_counts["neutral"] / total_items) * 100,
    }


def main():
    
    with open("reddit_data.json", "r") as f:
        data = json.load(f)

   
    sentiment_counts = Counter()
    for item in data:
        text_to_analyze = f"{item['title']} {item['content']}"  
        sentiment = analyze_sentiment_with_llm(text_to_analyze)
        sentiment_counts[sentiment] += 1

    
    total_items = len(data)
    sentiment_percentages = calculate_sentiment_percentages(sentiment_counts, total_items)

   
    print("Sentiment Analysis Results:")
    print(f"Bullish: {sentiment_percentages['bullish']:.2f}%")
    print(f"Bearish: {sentiment_percentages['bearish']:.2f}%")
    print(f"Neutral: {sentiment_percentages['neutral']:.2f}%")

   
    results = {
        "total_posts": total_items,
        "sentiment_percentages": sentiment_percentages,
        "sentiment_counts": dict(sentiment_counts)
    }
    with open("sentiment_results.json", "w") as f:
        json.dump(results, f, indent=2)


if __name__ == "__main__":
    main()